# Decorators in Python: Why and How to Use Them and Write Your Own
https://medium.com/swlh/decorators-in-python-why-and-how-to-use-them-and-write-your-own-c1da4ed9f3a9

Decorator is when you want to have the original functionality, but also every time you invoke that original functionality, you want something else to happen on top of it.  
Some examples of “tacked-on” software scenarios are:  
Caching  
Logging  
Access control  
Input validation  
Tweaks to input or output format

The task of optimizing the recursive Fibonacci number algorithm:

In [ ]:
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n-1) + fib(n-2)

fib(10)

55

The problem with this implementation is that you end up re-computing the same values repeatedly. For example, if you’re computing fib(10) that means fib(9) + fib(8). Then fib(9) = fib(8) + fib(7) and fib(8) = fib(7) + fib(6). Already you can see that fib(8) and fib(7) are being computed twice, and this repetition cascades through the rest of the recursion.  
The “known” way to optimize your Fibonacci solution is to add in caching, so that once fib(8) is calculated, you never have to calculate it again, you just have to retrieve it from the cache.

In [ ]:
cache = {}

def fib(n):
    if n < 2:
        return n

    if n in cache:
        return cache[n]

    result = fib(n-1) + fib(n-2)
    cache[n] = result
    return result

print(fib(10))
print(cache)

55
{2: 1, 3: 2, 4: 3, 5: 5, 6: 8, 7: 13, 8: 21, 9: 34, 10: 55}


Single-responsibility ❌  
Now instead of a function that computes Fibonacci numbers, we have a function that computes Fibonacci numbers and manages adding and retrieving things from a cache.  
Open-closed ❌  
We did not leave the existing function “closed”. If you looked at the Git blame for the function, 3 of the original lines of the function would be the same, 1 would be removed, and 6 would be added. You’ve re-written the majority of this whole function, and if you later decide to use a different kind of cache, you would need to “open up” the function again.

In general, the decorator approach means you want to “wrap” your code in this tacked-on functionality, meaning you create something that is composed of the original functionality, now decorated with the tacked-on functionality.  
Because we don’t need to modify the existing code, by using a decorator approach we are able to “tack on” the new functionality while still following the single-responsibility and open-closed SOLID principles.

In [ ]:
import functools

@functools.lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

fib(10)

55

Single-responsibility principle ✅  
The fib function, as written, is still just a Fibonacci number function. It is not a function combining Fibonacci numbers and caching.  
Open-closed principle ✅  
We left the original function “closed”. If someone went to look at the Git blame, it would still be fully attributed to the original author. But at the same time, we “opened” it up for extension, since now it’s using caching.  
Reality check: In case you are wondering if adding these two lines of code really “did anything”, I ran these two snippets using the %%timeit magic command and fib(25). The original code took 30.9 ms (30.9 milliseconds), and the code with caching took 6 µs (6 microseconds). There are 1000 microseconds in a millisecond, meaning the performance of the code with caching was over 1000x faster than the original—pretty impressive for an addition of 2 lines of code!

Conceptually, we “wrapped” the existing fib function in a cache decorator:  
1. The lru_cache decorator took in our function as an argument  
2. It added caching functionality  
3. It returned a new function composed of fib plus the caching logic  
4. Finally, it re-assigned the name fib to the new function, so we could continue using the original interface  

Using decorators that someone else has written is great! You can get some excellent functionality right “out of the box”: just import (if needed), add the @ symbol syntax, and your function is now decorated with added functionality.

Assigning functions to variables, and passing functions to other functions:

In [ ]:
def squared(num):
    return num**2

def cubed(num):
    return num**3

print(squared(5))
print(cubed(5))

25
125


The “first-class” functionality:

In [ ]:
def func_plus_two(polynomial_func, num):
    return polynomial_func(num) + 2

first_func = squared
second_func = cubed

print(func_plus_two(squared, 5))
print(func_plus_two(first_func, 5))
print(func_plus_two(second_func, 5))

27
27
127


**Nested functions**

Now instead of having squared and cubed as functions in the global scope, we have functions squared_inner and cubed_inner that only exist within the scope of the nth_degree function. In other words, if we tried to run squared_inner(5) in the global scope, it wouldn’t return 25, it would throw a NameError: name 'squared_inner' is not defined.

In [39]:
def nth_degree(num, n):
    def squared_inner(num):
        return num**2
    def cubed_inner(num):
        return num**3

    if n == 2:
        return squared_inner(num)
    elif n == 3:
        return cubed_inner(num)

print(nth_degree(5, 2))
print(nth_degree(5, 3))
print(nth_degree(5, 1))

25
125
None


**Writing our own comma-adding decorator.**  
“Our users are having a hard time reading these large numbers. At a glance, is 1000000 1 million or 10 million? Let’s take out the guesswork and put in some commas, like they’re used to in Comma Style format in Excel. So they see something like 1,000,000 instead of 1000000.”

In [ ]:
def add_commas(func):
    def add_commas_wrapper():
        # call original function
        # add in the commas
        # return the result

        original_string = func()
        # needs to be int for string formatting
        original_int = int(original_string)
        # we are ignoring locale, using default thousands sep
        return f'{original_int:,}'
    return add_commas_wrapper

In [ ]:
def one_million():
    return '1000000'

@add_commas
def one_billion():
    return '1000000000'

one_million = add_commas(one_million)
print(one_million)
print(type(one_million))
print(one_million())

print(one_billion)
print(type(one_billion))
print(one_billion())

<function add_commas.<locals>.add_commas_wrapper at 0x7fa390a63dd0>
<class 'function'>
1,000,000
<function add_commas.<locals>.add_commas_wrapper at 0x7fa390a63a70>
<class 'function'>
1,000,000,000


In [ ]:
original_int = 150000000
f'{original_int:,}'

'150,000,000'

### For PythonTutor  
https://pythontutor.com/visualize.html#mode=edit

In [ ]:
def add_commas(func):
    def add_commas_wrapper():
        
        original_string = func()
        original_int = int(original_string)
        
        return f'{original_int:,}'
    return add_commas_wrapper
    
def one_million():
    return '1000000'

@add_commas
def one_billion():
    return '1000000000'

one_million = add_commas(one_million)
print(one_million)
print(one_million())

print(one_billion)
print(one_billion())

# Python: декораторы функций и замыкания
https://www.youtube.com/watch?v=TdeZKWBppo4&t=0s

In [ ]:
import time

# Decorator function
def testTime(fn):
    def wrapper(*args, **kwargs):
        st = time.time()
        res = fn(*args, **kwargs)
        dt = time.time() - st
        print(f'Time of work: {dt} sec')
        return res
    return wrapper

# 1 function for test
def getNOD(a, b):
    while a != b:
        if a > b: a -= b
        else : b -= a
    return a

# 2 function for test
@testTime
def fastNOD(a, b):
    if a < b: a, b = b, a
    while b: a, b = b, a % b
    return a


test1 = testTime(getNOD)
test1(10000, 2)

res = fastNOD(10000, 2)
print(res)

Time of work: 0.0006451606750488281 sec
Time of work: 2.1457672119140625e-06 sec
2


1. Написать две функции создания списка из четных чисел от 0 до N (N - аргумент функции):  
[0, 2, 4, ..., N]  
с помощью метода append и с помощью инструмента list comprehensions (генератор списков). Через декоратор определить время работы этих функций.

In [18]:
import time

def time_check(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time() - start
        print(f'Time of work: {end} sec.')
        return res
    return wrapper

@time_check
def create_list_append(n):
    lst = []
    for i in range(n):
        if i % 2 == 0:
            lst.append(i)
    return lst

def create_list_comprehension(n):
    return [i for i in range(n) if i % 2 == 0]


print(create_list_append(10))

test_comprehension = time_check(create_list_comprehension)
print(test_comprehension(10))

Time of work: 4.0531158447265625e-06 sec.
[0, 2, 4, 6, 8]
Time of work: 3.5762786865234375e-06 sec.
[0, 2, 4, 6, 8]


### For PythonTutor  
https://pythontutor.com/visualize.html#mode=edit

In [16]:
def time_check(func):
    def wrapper(*args, **kwargs):        
        res = func(*args, **kwargs)        
        print(f'Hello.')
        return res
    return wrapper

@time_check
def create_list_append(n):
    lst = []
    for i in range(n):
        if i % 2 == 0:
            lst.append(i)
    return lst

test1 = time_check(create_list_append(10))

print(type(test1))
print(test1)

Hello.
<class 'function'>
<function time_check.<locals>.wrapper at 0x7ff9ecbf2c20>


2. Написать декоратор для кэширования результатов работы функции вычисления квадратного корня положительного целочисленого значения x. То есть, при повторном вызове функции (через декоратор) с одним и тем же аргументом, результат должен браться из кэша, а не вычисляться заново.  
Подсказка: следует использовать замыкание для хранения кэша.

In [50]:
cache = {}

def caching(func):
    def cache_wrapper(x):
        if x in cache:
            return cache[x]

        result = func(x)
        cache[x] = result

        return result
    return cache_wrapper

@caching
def sqroot(x):
    return x**(1/2)

print(sqroot(8))
print(cache)

2.8284271247461903
{8: 2.8284271247461903}


In [64]:
print(sqroot(9))
print(cache)

3.0
{8: 2.8284271247461903, 7: 2.6457513110645907, 2: 1.4142135623730951, 9: 3.0}


# Primer on Python Decorators  
https://realpython.com/primer-on-python-decorators/#first-class-objects  
https://dbader.org/blog/python-first-class-functions